In [3]:
import pandas as pd 

In [13]:
# LOAD THE DATA FROM crm ( BRONZE LEVEL )  

# REMOVE " # " TO RUN

# ------- CRM_CUST_INFO
# bronze_crm_cust_info = pd.read_csv('datasets/source_crm/cust_info.csv')
#-------- REMOVE " # " TO RUN 
# bronze_crm_cust_info

# ------- CRM_PRD_INFO
# bronze_crm_prd_info = pd.read_csv("datasets/source_crm/prd_info.csv")
#-------- REMOVE " # " TO RUN 
#bronze_crm_prd_info

# ------- CRM_sales_details
# bronze_crm_sales_details = pd.read_csv("datasets/source_crm/sales_details.csv")
#-------- REMOVE " # " TO RUN 
#bronze_crm_sales_details





TURN BRONZE DATA INTO SILVER
DATA TRANSFORMATION ( EDA )
- DATA CLEANING
- DATA NORMALIZATION
- MODIFY COLUMNS NAME ( FOR READABLITY )
- REMOVE AND REPLACE ALL NULL VALUES

In [ ]:
# --------- DATA TRANSFORMATION ( bronze_crm_cust_info )

# First step: check the data types
bronze_crm_cust_info.dtypes
#cst_id                float64
#cst_key                object
#cst_firstname          object
#cst_lastname           object
#cst_marital_status     object
#cst_gndr               object
#cst_create_date        object
#dtype: object

# REMOVE ALL NULL VALUES FROM " CST ID " 
bronze_crm_cust_info = bronze_crm_cust_info[~bronze_crm_cust_info['cst_id'].isnull()].reset_index(drop=True)



# REPLACE NAN VALUE TO ' N/A'
bronze_crm_cust_info.fillna('N/A',inplace=True)






# - CHANGE THE DATA TYPES OF create_date INTO DateTime and cst_id into INTEGER

bronze_crm_cust_info['cst_id'] = bronze_crm_cust_info['cst_id'].astype(int)

bronze_crm_cust_info['cst_create_date'] = pd.to_datetime(bronze_crm_cust_info['cst_create_date'])


# REMOVE ALL UNWANTED SPACES
bronze_crm_cust_info['cst_firstname'] = bronze_crm_cust_info['cst_firstname'].str.rstrip(" ")
bronze_crm_cust_info['cst_firstname'] = bronze_crm_cust_info['cst_firstname'].str.lstrip(" ")

bronze_crm_cust_info['cst_lastname'] = bronze_crm_cust_info['cst_lastname'].str.rstrip(" ")
bronze_crm_cust_info['cst_lastname'] = bronze_crm_cust_info['cst_lastname'].str.lstrip(" ")

#CHECKING THE REMAINING UNWANTED SPACES THAT HAVE ONE WORD VALUES
#bronze_crm_cust_info['cst_lastname'][bronze_crm_cust_info['cst_lastname'].str.contains(' ')]


# RENAMING COLUMNS 

bronze_crm_cust_info.columns
silver_crm_cust_info = bronze_crm_cust_info.rename(columns={'cst_id' : 'customer_id', 'cst_key':'customer_key', 'cst_lastname':'customer_lastname',
                            'cst_firstname':'customer_firstname', 'cst_marital_status': 'customer_marital_status', 'cst_gndr':'customer_gender',
                                   'cst_create_date':"customer_create_date"}
                          ).copy()
silver_crm_cust_info['customer_id'] = silver_crm_cust_info['customer_key'].str.slice(start=5, stop=12)
# ----------------------------- CLEAN! -----------------------------------# 
silver_crm_cust_info
# CONVERT INTO CSV FILES ------------
#silver_crm_cust_info.to_csv('silver_crm_cust_info.csv', index=False)


In [ ]:
#--------- DATA TRANSFORMATION ( bronze_crm_prd_info ) -----------#

#---- REMOVE PRD COST NaN Values   ( WE DONT WANT TO CONVERT NaN to N/A values on float&integer datatypes )

bronze_crm_prd_info = bronze_crm_prd_info[~bronze_crm_prd_info['prd_cost'].isnull()].reset_index(drop=True)

# REPLACE NAN VALUES TO 'N/A'
bronze_crm_prd_info.fillna('N/A',inplace = True)


# convert prd_start_dt to datetime
bronze_crm_prd_info['prd_start_dt'] = pd.to_datetime(bronze_crm_prd_info['prd_start_dt'])

bronze_crm_prd_info

#RENAMING COLUMNS
silver_crm_prd_info = bronze_crm_prd_info.rename(
    columns = {'prd_id' : 'product_id', 'prd_nm':'product_name', 'prd_cost':'product_cost',
              'prd_line':'product_line', 'prd_str_dt':'product_start_date', 'prd_end_dt':'product_end_date'
              }).copy()
silver_crm_prd_info



# -------------- CLEAN! ----------------#
# CONVERT INTO CSV FILES ------------
#silver_crm_prd_info.to_csv('silver_crm_prd_info.csv', index=False)




In [ ]:
#--------- DATA TRANSFORMATION & CLEANING ( bronze_crm_sales_details ) -----------#



 # CREATE ANOTHER COLUMNS THAT WE SEE OUR TOTAL SALES OF EACH ORDER ( SALES QUANTITY * SALES PRICE ) AND REMOVE SLS_SALES COLUMNS

# --------------- CREATING TOTAL  SALES COLUMNS ------------------------- # 
bronze_crm_sales_details['sls_total_sales'] = bronze_crm_sales_details['sls_quantity'] * bronze_crm_sales_details['sls_price']
bronze_crm_sales_details = bronze_crm_sales_details[~bronze_crm_sales_details['sls_total_sales'].isnull()]
bronze_crm_sales_details['sls_total_sales'].astype(int)

# ----------------- REMOVING SLS_SALES_COLUMNS ------------------------- #

bronze_crm_sales_details = bronze_crm_sales_details.drop(columns=['sls_sales'],axis=1)


# replace NaN to N/A
bronze_crm_sales_details.fillna('N/A',inplace=True)






#CONVERT sls_due_dt, sls_ship_dt and sls_order_dt to datetime
bronze_crm_sales_details['sls_ship_dt'] = pd.to_datetime(bronze_crm_sales_details['sls_ship_dt'])
bronze_crm_sales_details['sls_order_dt'] = pd.to_datetime(bronze_crm_sales_details['sls_order_dt'])
bronze_crm_sales_details['sls_due_dt'] = pd.to_datetime(bronze_crm_sales_details['sls_due_dt'])
bronze_crm_sales_details.head()


# ----------- RENAMING COLUMNS -----------------#
silver_crm_sales_details = bronze_crm_sales_details.rename(

    columns={'sls_ord_num':'sales_order_number','sls_prd_key':'sales_product_key','sls_cust_id':'sales_customer_id',
            'sls_order_dt':'sales_order_date','sls_ship_dt':'sales_ship_date', 'sls_due_dt':'sales_due_date', 'sls_quantity':'sales_quantity',
             'sls_price': 'sales_price','sls_total_sales':'sales_total_sales'
            }
).copy()


# ------------------------------- CLEAN! ------------------------------------------- #

# CONVERT INTO CSV FILES ------------
#silver_crm_sales_details.to_csv('silver_crm_sales_details.csv', index=False)
